In [1]:
import os
import sys

In [2]:
from tensorflow import keras;
import tensorflow as tf

In [3]:
import torch

In [4]:
import glob;
import math;
import numpy as np;
import time;


In [50]:
# !pip install git+https://github.com/alibaba/TinyNeuralNetwork.git

In [27]:
sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("../common/"))

In [68]:
from th.resources import calculator as calc
from th.resources import models as acdnet_torch_models
from datetime import datetime

In [51]:
from tinynn.converter import TFLiteConverter
from tinynn.graph.quantization.quantizer import PostQuantizer
from tinynn.graph.tracer import model_tracer
from tinynn.util.cifar10 import get_dataloader, calibrate
from tinynn.util.train_util import DLContext, get_device
from tinynn.graph.quantization.algorithm.cross_layer_equalization import cross_layer_equalize

In [ ]:
def genDataTimeStr():
    return datetime.today().strftime('%Y-%m-%d %H:%M:%S').replace('-',"").replace(' ',"").replace(':',"");

In [41]:
device = 'cpu'
keras_model_path = "./keras_models/retrain_pruned_cp_weights.h5"
retrain_src_model_path = "./retrained_models_after_pruned/retrained_model_20240124123209_acc_95.45455169677734_795th_epoch.pt"
quanted_torch_model_path = "../th/quantized_models/quant_retrained_model_94.3_20240124111811.pt"

In [9]:
keras_model = keras.models.load_model(keras_model_path)

In [10]:
keras_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 30225, 1)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 1, 15109, 5)       45        
                                                                 
 batch_normalization_12 (Ba  (None, 1, 15109, 5)       20        
 tchNormalization)                                               
                                                                 
 re_lu_12 (ReLU)             (None, 1, 15109, 5)       0         
                                                                 
 conv2d_13 (Conv2D)          (None, 1, 7553, 32)       800       
                                                                 
 batch_normalization_13 (Ba  (None, 1, 7553, 32)       128       
 tchNormalization)                                         

In [21]:
test_input = torch.randn((1, 1, 30225,1));
test_input = torch.Tensor.numpy(test_input)
test_input = tf.convert_to_tensor(test_input)

In [22]:
res = keras_model.predict(test_input)

1/1 [==============================] - 0s 9ms/step


In [20]:
print(type(res))

<class 'numpy.ndarray'>


In [29]:
state = torch.load(retrain_src_model_path, map_location=device)
config = state['config']
torch_weights = state['weight']

In [30]:
print(config)

[5, 32, 9, 16, 23, 33, 29, 56, 47, 65, 90, 2]


In [54]:
pruned_model = acdnet_torch_models.GetACDNetModel(nclass=2,channel_config=config)
quanted_model = acdnet_torch_models.GetACDNetQuantModel(nclass=2,channel_config=config)
quanted_model.load_state_dict(state['weight'])
pruned_model.load_state_dict(state['weight'])

<All keys matched successfully>

In [56]:
calc.summary(pruned_model, (1,1,30225));

+----------------------------------------------------------------------------+
+                           Pytorch Model Summary                            +
------------------------------------------------------------------------------
   Layer (type)       Input Shape      Output Shape    Param #      FLOPS #
       Conv2d-1     (1, 1, 30225)     (5, 1, 15109)         45      679,905
  BatchNorm2d-2     (5, 1, 15109)     (5, 1, 15109)         10            0
         ReLu-3     (5, 1, 15109)     (5, 1, 15109)          0       75,545
       Conv2d-4     (5, 1, 15109)     (32, 1, 7553)        800    6,042,400
  BatchNorm2d-5     (32, 1, 7553)     (32, 1, 7553)         64            0
         ReLu-6     (32, 1, 7553)     (32, 1, 7553)          0      241,696
    MaxPool2d-7     (32, 1, 7553)      (32, 1, 151)          0      241,600
      Permute-8      (32, 1, 151)      (1, 32, 151)          0            0
       Conv2d-9      (1, 32, 151)      (9, 32, 151)         81      391,392
 Ba

# TinyNeuralNetwork provides a PostQuantizer class that may rewrite the graph for and perform model fusion for
        # quantization. The model returned by the `quantize` function is ready for quantization calibration.
        # By default, the rewritten model (in the format of a single file) will be generated in the working directory.
        # You may also pass some custom configuration items through the argument `config` in the following line. For
        # example, if you have a quantization-rewrite-ready model (e.g models in torchvision.models.quantization),
        # then you may use the following line.
        #   quantizer = PostQuantizer(model, dummy_input, work_dir='out', config={'rewrite_graph': False})
        # Alternatively, if you have modified the generated model description file and want the quantizer to load it
        # instead, then use the code below.
        #     quantizer = PostQuantizer(
        #         model, dummy_input, work_dir='out', config={'force_overwrite': False, 'is_input_quantized': None}
        #     )
        # The `is_input_quantized` in the previous line is a flag on the input tensors whether they are quantized or
        # not, which can be None (False for all inputs) or a list of booleans that corresponds to the inputs.
        # Also, we support multiple qschemes for quantization preparation. There are several common choices.
        #   a. Asymmetric uint8. (default) config={'asymmetric': True, 'per_tensor': True}
        #      The is the most common choice and also conforms to the legacy TFLite quantization spec.
        #   b. Asymmetric int8. config={'asymmetric': True, 'per_tensor': False}
        #      The conforms to the new TFLite quantization spec. In legacy TF versions, this is usually used in post
        #      quantization. Compared with (a), it has support for per-channel quantization in supported kernels
        #      (e.g Conv), while (a) does not.
        #   c. Symmetric int8. config={'asymmetric': False, 'per_tensor': False}
        #      The is same to (b) with no offsets, which may be used on some low-end embedded chips.
        #   d. Symmetric uint8. config={'asymmetric': False, 'per_tensor': True}
        #      The is same to (a) with no offsets. But it is rarely used, which just serves as a placeholder here.
        # In addition, we support additional ptq algorithms including kl-divergence, the usage is shown as below:
        #       quantizer = PostQuantizer(model, dummy_input, work_dir='out', config={'algorithm': 'kl'})

In [62]:
torch_dummy_input = torch.randn((1, 1, 1, 30225));
quant_config = {'asymmetric': True, 'per_tensor': False, 'rewrite_graph': False, 'force_overwrite': False,
                'is_input_quantized': None, 'algorithm': 'kl'}

In [63]:
quantizer = PostQuantizer(pruned_model, dummy_input=torch_dummy_input, work_dir="../th/quantized_models/", config=quant_config)
        #         model, dummy_input, work_dir='out', config={'force_overwrite': False, 'is_input_quantized': None}
        #     )

In [64]:
ptq_model = quantizer.quantize()

In [67]:
device = 'cpu'
ptq_model.to(device=device)

ACDNetV2(
  (sfeb): Sequential(
    (0): Conv2d(1, 5, kernel_size=(1, 9), stride=(1, 2), bias=False)
    (1): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(5, 32, kernel_size=(1, 5), stride=(1, 2), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(1, 50), stride=(1, 50), padding=0, dilation=1, ceil_mode=False)
  )
  (tfeb): Sequential(
    (0): Conv2d(1, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(9, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Conv2d(16, 23, 

In [ ]:
context = DLContext()
context.device = 'cpu'
context.train_loader, context.val_loader = get_dataloader(args.data_path, 224, args.batch_size, args.workers)
context.max_iteration = 100

# Post quantization calibration
calibrate(ptq_model, context)

with torch.no_grad():
    ptq_model.eval()
    ptq_model.cpu()

    # The step below converts the model to an actual quantized model, which uses the quantized kernels.
    ptq_model = quantizer.convert(ptq_model)

    # When converting quantized models, please ensure the quantization backend is set.
    torch.backends.quantized.engine = quantizer.backend

    # The code section below is used to convert the model to the TFLite format
    # If you need a quantized model with a specific data type (e.g. int8)
    # you may specify `quantize_target_type='int8'` in the following line.
    # If you need a quantized model with strict symmetric quantization check (with pre-defined zero points),
    # you may specify `strict_symmetric_check=True` in the following line.
    converter = TFLiteConverter(ptq_model, dummy_input, tflite_path='../th/quantized_models/final_quanted_model_{}'.format(gen))
    converter.convert()


In [57]:
# torch.quantization.get_default_qconfig('qnnpack')
# quanted_torhch_state = torch.jit.load(quanted_torch_model)

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.HistogramObserver'>, reduce_range=False){}, weight=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric){})